In [1]:
import io
import requests
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.widgets import Slider, Button, RadioButtons
import seaborn as sns
import PIL.Image as Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn import metrics

pd.set_option('display.max_columns', None) # 针对数据集中列比较多的情况，把dataframe中的列全部显示出来

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 6.0)  # set default size of plots
#plt.rcParams['image.aspect'] = 1.3
plt.rcParams['image.interpolation'] = 'nearest' # 设置 interpolation style
#plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['savefig.dpi'] = 300  #图片像素
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.unicode_minus'] = False          # 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['lines.markersize'] = 8
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False #display only a left and bottom box border
plt.rcParams['legend.edgecolor'] = 'white'
plt.rcParams['axes.axisbelow'] = False
plt.rcParams['grid.color'] = 'white'

# 朴素贝叶斯

## 获取文本数据集

In [2]:
import zipfile
r = requests.get('https://github.com/cystanford/text_classification/archive/master.zip',stream = True)
print(r.status_code)
f = io.BytesIO(r.content)
#with zipfile.ZipFile(f) as myzip:
#    myzip.extractall(path='/Users/mymac/Documents/Task/pandas/lahman-csv_2014-02-14')
names = zipfile.ZipFile(f).namelist()
print(names[0:5])

200
['text_classification-master/', 'text_classification-master/README.md', 'text_classification-master/text classification/', 'text_classification-master/text classification/stop/', 'text_classification-master/text classification/stop/stopword.txt']


In [3]:
def read_txt(filename, encoding='utf-8'):
    with zipfile.ZipFile(f) as myzip:
        with myzip.open(filename, 'r') as myfile:
            lines = myfile.readlines()

    # print(filename)
    stop_words = [line.strip().decode(encoding, 'ignore')
                  for line in lines]  #'ignore'
    return stop_words


stop_words_2 = read_txt(
    'text_classification-master/text classification/stop/stopword.txt',
    encoding='utf-8-sig')
print(' '.join([x for x in stop_words_2][0:100]))

, ? 、 。 “ ” 《 》 ！ ， ： ； ？ 人民 # ### 啊 阿 哎 哎呀 哎哟 唉 俺 俺们 按 按照 吧 吧哒 把 罢了 被 本 本着 比 比方 比如 鄙人 彼 彼此 边 别 别的 别说 并 并且 不比 不成 不单 不但 不独 不管 不光 不过 不仅 不拘 不论 不怕 不然 不如 不特 不惟 不问 不只 朝 朝着 趁 趁着 乘 冲 除 除此之外 除非 除了 此 此间 此外 从 从而 打 待 但 但是 当 当着 到 得 的 的话 等 等等 地 第 叮咚 对 对于 多 多少 而 而况 而且


In [4]:
from itertools import chain

def read_folder(reg_str):
    folder_names = list(filter(re.compile(reg_str).match, names))
    print(folder_names)
    data = []
    for ifolder_name in folder_names:
        data_files = list(filter(re.compile(ifolder_name + ".+$").match, names))
        # print(data_files[0])
        idata = [read_txt(idata_file, encoding='gbk') for idata_file in data_files]
        data.append(list(chain(*idata)))
        
    return data

In [5]:
train = read_folder(".*train/.+/$")
test = read_folder(".*test/.+/$")
print(train[0][:5])
print('_' * 80)
print(test[0][:5])

['text_classification-master/text classification/train/体育/', 'text_classification-master/text classification/train/女性/', 'text_classification-master/text classification/train/文学/', 'text_classification-master/text classification/train/校园/']
['text_classification-master/text classification/test/体育/', 'text_classification-master/text classification/test/女性/', 'text_classification-master/text classification/test/文学/', 'text_classification-master/text classification/test/校园/']
['球场禁用招数“少先队员之惩戒”冠', '可以直接到编辑部买，地址，北京体育馆路8号，中国体育报业总社院内，后楼51700：羽毛球杂志木有支付宝，木有财付通，在网上订购不支持货到付款么？那么北京哪个实体店有卖12月的《羽毛球》杂志，或者说，能去你们编辑社买不？地址？转发(3)评论(4)12月10日09:17来自新浪微博', '直播贴阿内尔卡正式加盟申花，最现场连线申花新闻官带来第一手消息，上海男篮2分惜败惨遭4连败，北京延续不败战绩欧洲足坛激战正酣，米兰被弱旅逼平，拜仁重夺联赛榜首风云变幻，霍华德要求离队，湖人退出三方交易更多内容请听11:00的《990体育新闻》934:', '组图：金妍儿黑丝亮相热心公益 OL套装透成熟 http:url.cn/1bNQDP  (分享自 腾讯体育 )$LOTOzf$', '北京23分落后末节大反扑 惜败佛山终结13连胜  http:url.cn/2nGdrq  (分享自 腾讯体育 )$LOTOzf$']
________________________________________________________________________________
['意大利的新人Edoard

## 整理数据集

In [6]:
import jieba

# 分类label
categories = ['sports', 'female', 'literature', 'campus']

def get_labels(data):
    data_labels = [[icategory] * len(idata)
                   for idata, icategory in zip(data, categories)]
    return list(chain(*data_labels))

train_labels = get_labels(train)
test_labels = get_labels(test)
print(train_labels[:5])
print('_' * 80)
print(test_labels[:5])

# 分类features
def get_features(data):
    data_chain = list(chain(*data))
    # 分词 & 去掉非中文字符
    data_chain = [
        re.compile(u'[^\u4E00-\u9FFF]').sub(
            r' ', ' '.join(jieba.cut_for_search(x))).strip()
        for x in data_chain
    ]
    print(data_chain[:5])
    print('_' * 80)
    return data_chain

train_chain = get_features(train)
test_chain = get_features(test)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8b/mn7p_n410l5dh_8lxtjy_m680000gn/T/jieba.cache


['sports', 'sports', 'sports', 'sports', 'sports']
________________________________________________________________________________
['sports', 'sports', 'sports', 'sports', 'sports']


Loading model cost 0.870 seconds.
Prefix dict has been built succesfully.


['球场 禁用 招数   队员 少先队 少先队员 之 惩戒   冠', '可以 直接 到 编辑 编辑部 买   地址   北京 体育 体育馆 路   号   中国 体育 报业 总社 院内   后楼         羽毛 羽毛球 杂志 木有 支付 支付宝   木有 财付 通   在 网上 订购 不 支持 付款 货到付款 么   那么 北京 哪个 实体 实体店 有 卖    月 的   羽毛 羽毛球   杂志   或者 或者说   能 去 你们 编辑 社买 不   地址   转发       评论          月    日         来自 新浪 微博', '直播 贴 阿内 内尔 阿内尔 阿内尔卡 正式 加盟 申花   最 现场 连线 申花 新闻 新闻官 带来 第一 一手 第一手 消息   上海 男篮   分 惜败 惨遭   连败   北京 延续 不败 战绩 欧洲 足坛 激战 正酣   米兰 被 弱旅 逼平   拜仁 重夺 联赛 榜首 风云 变幻 风云变幻   霍华德 要求 离队   湖人 退出 三方 交易 更 多 内容 请 听         的       体育 育新 新闻 体育新闻', '组图   金妍儿 黑丝 亮相 热心 公益      套装 透 成熟                                  分享 自   腾讯 体育', '北京    分 落后 末节 大 反扑   惜败 佛山 终结    连胜                                    分享 自   腾讯 体育']
________________________________________________________________________________
['大利 意大利 的 新人                   上周 周末 上周末 成为     冠军 俱乐部 的 一员   他 在     威尼 尼斯 威尼斯 站点 的 比赛 获得 了 冠军 和 奖金      万 欧元', '李娜 真棒   坚持 到 了 最后   国歌 在 法网 比赛 中 第一 一次 第一次 响起   好 激动 哈', '北京 时间   月   日   凯尔 尔特 凯尔特 凯尔特人 队 消息   前锋   大 宝贝   格伦   维斯 戴维斯 将 因为 左膝

## 提取特征

In [7]:
# 去掉新浪|微博|转发|评论|分享|腾讯|标签|原文
train_chain = [re.compile(u'新浪|微博|转发|评论|分享|腾讯').sub(r'', x).strip() for x in train_chain]
test_chain = [re.compile(u'新浪|微博|转发|评论|分享|腾讯').sub(r'', x).strip() for x in test_chain]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec_train = TfidfVectorizer(
    stop_words=stop_words_2, max_df=0.5, analyzer='word') #, ngram_range=(1,2)
tfidf_matrix_train = tfidf_vec_train.fit_transform(train_chain)
print(tfidf_vec_train.get_feature_names()[:30])
train_features = tfidf_matrix_train.toarray()
print(train_features.shape)
#print(train_features[:5])

train_vocabulary = tfidf_vec_train.vocabulary_
tfidf_vec_test = TfidfVectorizer(
    stop_words=stop_words_2, max_df=0.5, vocabulary=train_vocabulary)
tfidf_matrix_test = tfidf_vec_test.fit_transform(test_chain)
print(tfidf_vec_train.get_feature_names()[:30])
test_features = tfidf_matrix_test.toarray()
print(test_features.shape)
#print(test_features[:5])

/Users/mymac/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['一一', '一丁', '一万', '一万元', '一上', '一上午', '一下', '一下下', '一下子', '一下脸', '一丝', '一个个', '一个伍', '一个名', '一个多', '一个多月', '一个月', '一个男孩', '一中', '一串', '一为', '一举', '一举一动', '一举三得', '一举两得', '一书', '一事', '一事无成', '一二', '一二个']
(3313, 23507)
['一一', '一丁', '一万', '一万元', '一上', '一上午', '一下', '一下下', '一下子', '一下脸', '一丝', '一个个', '一个伍', '一个名', '一个多', '一个多月', '一个月', '一个男孩', '一中', '一串', '一为', '一举', '一举一动', '一举三得', '一举两得', '一书', '一事', '一事无成', '一二', '一二个']
(200, 23507)


/Users/mymac/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## 检查特征

In [9]:
tfidfs = pd.concat([
    pd.DataFrame(train_features),
    pd.DataFrame(train_labels, columns=['label'])
],
                   axis=1)
tfidfs_sum = tfidfs.groupby('label').sum()
voc = pd.Series(tfidf_vec_train.vocabulary_)
print(tfidfs_sum.index.tolist())
print('_' * 80)
for i in np.arange(4):
    print(voc[voc.isin(
        pd.Series(tfidfs_sum.iloc[i, ].sort_values(
            ascending=False)[:30].index))].index.tolist())
    print('_' * 80)

['campus', 'female', 'literature', 'sports']
________________________________________________________________________________
['举行', '标签', '参加', '业生', '教育', '报名', '大学', '学生', '孩子', '老师', '毕业', '毕业生', '招聘', '同学', '原文', '青年', '保护', '绿色', '学院', '小学', '校园', '大学生', '考试', '环保', '就业', '校车', '研究生', '本科', '招生', '考生']
________________________________________________________________________________
['分钟', '喜欢', '收听', '每天', '原文', '妈妈', '化妆', '减肥', '肌肤', '女人', '美丽', '美容', '注射', '除皱', '蛋白', '皮肤', '整形', '女性', '星座', '化妆品', '面膜', '瘦身', '淘宝', '保湿', '美白', '蜂蜜', '柠檬', '护肤', '指甲', '美甲']
________________________________________________________________________________
['编辑', '杂志', '标签', '敬请', '周刊', '不错', '原文', '布斯', '阅读', '故事', '作家', '封面', '读者', '上市', '月刊', '图书', '读书', '火花', '小说', '出版社', '新书', '好书', '意林', '一本', '文学', '本书', '订阅', '乔布', '乔布斯', '想读']
________________________________________________________________________________
['体育', '羽毛', '羽毛球', '直播', '阿内', '内尔', '阿内尔', '阿内尔卡', '联赛', '风云', '科比', '皇马', '冠军',

In [10]:
tfidfs_sum[voc[[x == u'报名' for x in voc.index.tolist()]]]
tfidfs_sum[voc[[x == u'招生' for x in voc.index.tolist()]]]
tfidfs_sum[voc[[x == u'论文' for x in voc.index.tolist()]]]
tfidfs_sum[voc[[x == u'毕业' for x in voc.index.tolist()]]]

,13819
label,
campus,3.884964
female,0.000000
literature,0.225690
sports,0.576862


## 多项式贝叶斯分类器

In [11]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB  
model = MultinomialNB(alpha=0.01).fit(train_features, train_labels)

prediction = model.predict(test_features)
print('The accuracy of the MultinomialNB is',
      metrics.accuracy_score(prediction, test_labels))
print(metrics.classification_report(test_labels, prediction))
print(metrics.confusion_matrix(test_labels, prediction))

The accuracy of the MultinomialNB is 0.92
              precision    recall  f1-score   support

      campus       0.93      0.81      0.87        16
      female       0.97      0.87      0.92        38
  literature       0.71      0.97      0.82        31
      sports       0.98      0.94      0.96       115

   micro avg       0.92      0.92      0.92       200
   macro avg       0.90      0.90      0.89       200
weighted avg       0.93      0.92      0.92       200

[[ 13   0   3   0]
 [  0  33   4   1]
 [  0   0  30   1]
 [  1   1   5 108]]
